In [3]:
# 导入需要的库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter


ModuleNotFoundError: No module named 'numpy.random.bit_generator'

In [ ]:
data = pd.read_csv('data.csv') 

X = data.iloc[:,1:-1]  
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
print(X_train)
print(y_train)
print(X_test)
print(y_test)

NameError: name 'train_test_split' is not defined

In [ ]:
# 信息熵
def calc_entropy(y):
    y = pd.Series(y) 
    counts = y.value_counts()
    # #打印每个类别的数量
    # print(counts)
    ps = counts / len(y)
    # print(ps)
    return -ps.map(lambda p: p * np.log2(p)).sum()

#处理连续值
def calc_entropy2(y,spilt_point):
    # print(y)
    y = pd.Series(y)
    y1=y[y<=spilt_point]
    y2=y[y>spilt_point]    
    ps1 = len(y1) / len(y)
    ps2 = len(y2) / len(y)
    return -(ps1*np.log2(ps1)+ps2*np.log2(ps2))


# 计算条件熵
def cond_entropy(X, y):
    y = pd.Series(y)
    h_total = 0
    # print(X.unique(),len(X.unique()))
    for x in X.unique():
        mask = X.isin([x]) & ~X.isna() # 选出X中等于x的样本
        h_x = calc_entropy(y[mask]) # 计算条件熵
        p_x = mask.mean() # 计算p(x)
        h_total += p_x * h_x
        # print(x, p_x, h_x)
    return h_total



# #打印信息熵
# print("信息熵：",calc_entropy(y))
# #打印条件熵
# print("色泽：",cond_entropy(X['色泽'], y))
# print("根蒂：",cond_entropy(X['根蒂'], y))
# print("敲声：",cond_entropy(X['敲声'], y))
# print("纹理：",cond_entropy(X['纹理'], y))
# print("脐部：",cond_entropy(X['脐部'], y))
# print("触感：",cond_entropy(X['触感'], y))

In [ ]:
#对于连续值的处理
def generate_split_points(data, attribute):
    unique_values = set(data[attribute])  # 获取属性的唯一取值
    split_points = []  # 存储候选分割点
    for i in range(len(unique_values) - 1):
        current_value = sorted(list(unique_values))[i]
        next_value = sorted(list(unique_values))[i + 1]
        split_point = (current_value + next_value) / 2  # 取相邻两个值的中点作为分割点
        split_points.append(split_point)
    return split_points

# print(generate_split_points(X, '密度'))
# print(generate_split_points(X, '含糖率'))

#选出最优的分割点
def choose_best_split_point(data, attribute, y):
    split_points = generate_split_points(data, attribute)
    min_entropy = 1
    best_split_point = None
    for split_point in split_points:
        mask = data[attribute] <= split_point
        h = cond_entropy(pd.Series(mask), y)
        if h < min_entropy:
            min_entropy = h
            best_split_point = split_point
    info_gain = calc_entropy(y) - min_entropy
    return best_split_point, info_gain

def choose_best_split_point2(data, attribute, y):
    split_points = generate_split_points(data, attribute)
    min_entropy = 1
    best_split_point = None
    for split_point in split_points:
        mask = data[attribute] <= split_point
        h = cond_entropy(pd.Series(mask), y)
        if h < min_entropy:
            min_entropy = h
            best_split_point = split_point
    info_gain = calc_entropy(y) - min_entropy
    Info_gain_rate = info_gain / calc_entropy2(data[attribute],best_split_point)
    return best_split_point, Info_gain_rate
# print(choose_best_split_point(X, '密度', y))
# print(choose_best_split_point(X, '含糖率', y))

In [ ]:
def Info_gain(X, y):
    return calc_entropy(y) - cond_entropy(X, y)

#信息增益率
def Info_gain_rate(X, y):
    return Info_gain(X, y) / calc_entropy(X)


# #打印每个特征的信息增益
# print("色泽：",Info_gain(X['色泽'], y))
# print("根蒂：",Info_gain(X['根蒂'], y))
# print("敲声：",Info_gain(X['敲声'], y))
# print("纹理：",Info_gain(X['纹理'], y))
# print("脐部：",Info_gain(X['脐部'], y))
# print("触感：",Info_gain(X['触感'], y))
# #打印连续值信息最佳分割点以及信息增益
# print("密度：",choose_best_split_point(X, '密度', y))
# print("含糖率：",choose_best_split_point(X, '含糖率', y))


In [ ]:
#定义决策树节点
label_count = 0 
class DecisionTreeNode:
    def __init__(self):
        global label_count
        self.attribute = None # 存储属性
        self.split_point = None # 存储分割点
        self.result = None # 存储叶节点的类别
        self.children = [] # 存储子节点
        self.branchvaule = None #分支的值
        self.label = label_count #用于广度优先遍历
        self.count = 0 #用于可视化所有节点
        self.fake_result = None # 用于后剪枝,存储子节点中样本数最多的类别
        
    def add_count(self):#分支的值
        self.count +=1    
        
    def set_branchvalue(self,branchvaule):#分支的值
        self.branchvaule = branchvaule

    def set_attribute(self, attribute):
        self.attribute = attribute

    def set_split_point(self, split_point):
        self.split_point = split_point

    def set_result(self, result):
        self.result = result

    def set_child(self, child):
        self.children.append(child)

    def set_fake_result(self, fake_result):
        self.fake_result = fake_result

    def is_leaf(self):
        return self.result is not None

预剪枝

In [ ]:

# 计算准确率
def accuracy(y_true, y_pred):
    # print(y_true,y_pred)
    return np.sum(y_true == y_pred) / len(y_true)


#寻找最佳属性
def find_best_attribute(X, y):
    best_info_gain = -np.inf
    best_attribute = None
    best_split_point = None
    for attribute in X.columns:
        if X[attribute].dtype == np.float64 or X[attribute].dtype == np.int32:  # 为浮点或整数类型，处理连续属性
            split_point, info_gain = choose_best_split_point(X, attribute, y)
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_attribute = attribute
                best_split_point = split_point
        else:  # 处理离散属性
            info_gain = Info_gain(X[attribute], y)
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_attribute = attribute
                best_split_point = None
    # #找到后将该属性从X中删除
    # X.drop(best_attribute,axis=1,inplace=True)
    return best_attribute,best_split_point,best_info_gain

#寻找最佳属性
def find_best_attribute2(X, y):
    best_info_gain = -np.inf
    best_attribute = None
    best_split_point = None
    for attribute in X.columns:
        if X[attribute].dtype == np.float64:  # 处理连续属性
            split_point, info_gain = choose_best_split_point2(X, attribute, y)
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_attribute = attribute
                best_split_point = split_point
        else:  # 处理离散属性
            info_gain = Info_gain_rate(X[attribute], y)
            if info_gain > best_info_gain:
                best_info_gain = info_gain
                best_attribute = attribute
                best_split_point = None
    return best_attribute,best_split_point,best_info_gain

#判断是否预剪枝
def is_pre_pruning(X,y,X_valid, y_valid,best_attribute,best_split_point,node):
    #预剪枝,验证集不为空且所有样本不属于同一类别
    if X_valid is not None:
        
        #计算划分前的准确率
        y_pred_pre = Counter(y).most_common(1)[0][0]
        print("划分前的结果：",y_pred_pre)
        accuracy_pre = accuracy(y_valid, [y_pred_pre]*len(y_valid))
        print("划分前的准确率：",accuracy_pre)
        #计算划分后的准确率
        y_pred_post = []
        #是否为连续值属性
        if best_split_point is None:  
            # 处理离散属性
            for value in X[best_attribute].unique():
                mask = X[best_attribute] == value
                mask_valid = X_valid[best_attribute] == value
                y_subset = y[mask]
                #验证集中的样本
                X_valid_subset = X_valid[mask_valid]
                y_valid_subset = y_valid[mask_valid]
                
                # 统计子集中最多类别得出该分支的类别
                y_pred_post_bool = Counter(y_subset).most_common(1)[0][0]
                # 将最多类别重复len(y_subset)次添加到列表
                y_pred_post += [y_pred_post_bool] * len(y_valid_subset)
                
            y_pred_post = np.array(y_pred_post)
            print(y_pred_post)
            accuracy_post = accuracy(y_valid, y_pred_post)
            print("离散属性划分后的准确率：",accuracy_post)
        else: # 处理连续属性
            mask = X[best_attribute] <= best_split_point
            mask_valid = X_valid[best_attribute] <= best_split_point
            # 选出X中小于等于best_split_point的样本划分子树
            y_left = y[mask]
            X_valid_left = X_valid[mask_valid]
            y_valid_left = y_valid[mask_valid]
            
            
            # 选出X中大于best_split_point的样本划分子树
            y_right = y[~mask]
            X_valid_right = X_valid[~mask_valid]
            y_valid_right = y_valid[~mask_valid]
            
            # 统计左子集中最多类别
            y_pred_post_left = Counter(y_left).most_common(1)[0][0]
            # 将最多类别重复len(y_subset)次添加到列表
            y_pred_post += [y_pred_post_left] * len(y_valid_left)
            
            # 统计左子集中最多类别
            y_pred_post_right = Counter(y_right).most_common(1)[0][0]
            # 将最多类别重复len(y_subset)次添加到列表
            y_pred_post += [y_pred_post_right] * len(y_valid_right)
            
            
            y_pred_post = np.array(y_pred_post)
            print(y_pred_post)
            accuracy_post = accuracy(y_valid, y_pred_post)
            print("连续属性划分后的准确率：",accuracy_post)
        #如果划分后的准确率小于划分前的准确率，创建叶子节点，选择最多的类别作为结果
        if accuracy_post <= accuracy_pre:
            # print("剪枝")
            # node = DecisionTreeNode()
            # node.set_result(y_pred_pre)
            return True
        else:
            print("不剪枝")
            return False
    else:
        print("没有验证集，不剪枝")
        return False
    
# 决策树代码
def build_decision_tree(X, y):
    global label_count 
    if len(set(y)) == 1:  # 所有样本属于同一类别，创建叶子节点
        leaf_node = DecisionTreeNode()
        leaf_node.set_result(y.iloc[0])
        return leaf_node
    
    if len(X.columns) == 0:  # 所有属性已经用完，创建叶子节点，选择最多的类别作为结果
        leaf_node = DecisionTreeNode()
        most_common_label = y.value_counts().idxmax()
        leaf_node.set_result(most_common_label)
        return leaf_node
    
    # 选取最佳划分属性
    best_info_gain = -np.inf
    best_attribute = None
    best_split_point = None
    best_attribute,best_split_point,best_info_gain = find_best_attribute(X, y)
    if best_info_gain < 0:  # 信息增益小于0，创建叶子节点，选择最多的类别作为结果
        leaf_node = DecisionTreeNode()
        most_common_label = y.value_counts().idxmax()
        leaf_node.set_result(most_common_label)
        return leaf_node
    
    node = DecisionTreeNode()
    node.set_attribute(best_attribute)
    node.set_split_point(best_split_point)
    node.add_count()
    node.set_fake_result(y.value_counts().idxmax())
    
    if not node.is_leaf():
        label_count += 1
        
    print(X,y)
    print("当前最佳划分属性：",node.attribute,"当前标签值：",node.label,"当前假设结果：",node.fake_result)
    
    # 递归构建子树
    if best_split_point is None:  # 处理离散属性
        for value in X[best_attribute].unique():
            mask = X[best_attribute] == value# 选出X中等于value的样本划分子树
            X_subset = X[mask]
            y_subset = y[mask]
            if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                leaf_node = DecisionTreeNode()
                most_common_label = y.value_counts().idxmax()#返回最多的类别
                leaf_node.set_result(most_common_label)#设置叶子节点的结果
                leaf_node.set_branchvalue(value)
                print(leaf_node.branchvaule,0)
                node.set_child(leaf_node)#设置子树
            else:
                childnode = build_decision_tree(X_subset, y_subset)#递归创建左子树
                childnode.set_branchvalue(value)
                print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                node.set_child(childnode)#递归创建子树
    else:  # 处理连续属性
            #for循环分别处理大于和小于两个子树
        for num in range(2):
            if num == 0:
                #小于等于分割点的部分
                mask = X[best_attribute] <= best_split_point
                X_subset = X[mask]
                y_subset = y[mask]
                if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                    leaf_node = DecisionTreeNode()
                    most_common_label = y.value_counts().idxmax()
                    leaf_node.set_result(most_common_label)
                    leaf_node.set_branchvalue('<='+str(best_split_point))
                    # print(leaf_node.branchvaule)
                    node.set_child(leaf_node)
                    print('无样本，判断为',most_common_label)
                else:
                    print(node.attribute,'小于等于',node.split_point,'样本个数：',len(X_subset))
                    childnode = build_decision_tree(X_subset, y_subset)
                    childnode.set_branchvalue('<='+str(best_split_point))
                    print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                    node.set_child(childnode)
            if num == 1:
                #大于分割点的部分
                mask = X[best_attribute] > best_split_point
                X_subset = X[mask]
                y_subset = y[mask]    
                if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                    leaf_node = DecisionTreeNode()
                    most_common_label = y.value_counts().idxmax()
                    leaf_node.set_result(most_common_label)
                    leaf_node.set_branchvalue('>'+str(best_split_point))
                    print(leaf_node.branchvaule,leaf_node.result)
                    node.set_child(leaf_node)
                    print('无样本，判断为',most_common_label)
                else:
                    print(node.attribute,'大于',node.split_point,'样本个数：',len(X_subset))
                    childnode = build_decision_tree(X_subset, y_subset)
                    childnode.set_branchvalue('>'+str(best_split_point))
                    print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                    node.set_child(childnode)
    return node

#信息增益率决策树代码
def build_decision_tree_rate(X, y):
    global label_count 
    if len(set(y)) == 1:  # 所有样本属于同一类别，创建叶子节点
        leaf_node = DecisionTreeNode()
        leaf_node.set_result(y.iloc[0])
        return leaf_node
    
    if len(X.columns) == 0:  # 所有属性已经用完，创建叶子节点，选择最多的类别作为结果
        leaf_node = DecisionTreeNode()
        most_common_label = y.value_counts().idxmax()
        leaf_node.set_result(most_common_label)
        return leaf_node
    
    # 选取最佳划分属性
    best_info_gain_rate = -np.inf
    best_attribute = None
    best_split_point = None
    best_attribute,best_split_point,best_info_gain_rate = find_best_attribute2(X, y)
    if best_info_gain_rate < 0:  # 信息增益小于0，创建叶子节点，选择最多的类别作为结果
        leaf_node = DecisionTreeNode()
        most_common_label = y.value_counts().idxmax()
        leaf_node.set_result(most_common_label)
        return leaf_node
    
    node = DecisionTreeNode()
    node.set_attribute(best_attribute)
    node.set_split_point(best_split_point)
    node.add_count()
    node.set_fake_result(y.value_counts().idxmax())
    
    if not node.is_leaf():
        label_count += 1
        
    print(X,y)
    print("当前最佳划分属性：",node.attribute,"当前标签值：",node.label,"当前假设结果：",node.fake_result)
    
    # 递归构建子树
    if best_split_point is None:  # 处理离散属性
        for value in X[best_attribute].unique():
            mask = X[best_attribute] == value# 选出X中等于value的样本划分子树
            X_subset = X[mask]
            y_subset = y[mask]
            if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                leaf_node = DecisionTreeNode()
                most_common_label = y.value_counts().idxmax()#返回最多的类别
                leaf_node.set_result(most_common_label)#设置叶子节点的结果
                leaf_node.set_branchvalue(value)
                print(leaf_node.branchvaule,0)
                node.set_child(leaf_node)#设置子树
            else:
                childnode = build_decision_tree(X_subset, y_subset)#递归创建左子树
                childnode.set_branchvalue(value)
                print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                node.set_child(childnode)#递归创建子树
    else:  # 处理连续属性
            #for循环分别处理大于和小于两个子树
        for num in range(2):
            if num == 0:
                #小于等于分割点的部分
                mask = X[best_attribute] <= best_split_point
                X_subset = X[mask]
                y_subset = y[mask]
                if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                    leaf_node = DecisionTreeNode()
                    most_common_label = y.value_counts().idxmax()
                    leaf_node.set_result(most_common_label)
                    leaf_node.set_branchvalue('<='+str(best_split_point))
                    # print(leaf_node.branchvaule)
                    node.set_child(leaf_node)
                    print('无样本，判断为',most_common_label)
                else:
                    print(node.attribute,'小于等于',node.split_point,'样本个数：',len(X_subset))
                    childnode = build_decision_tree(X_subset, y_subset)
                    childnode.set_branchvalue('<='+str(best_split_point))
                    print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                    node.set_child(childnode)
            if num == 1:
                #大于分割点的部分
                mask = X[best_attribute] > best_split_point
                X_subset = X[mask]
                y_subset = y[mask]    
                if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                    leaf_node = DecisionTreeNode()
                    most_common_label = y.value_counts().idxmax()
                    leaf_node.set_result(most_common_label)
                    leaf_node.set_branchvalue('>'+str(best_split_point))
                    print(leaf_node.branchvaule,leaf_node.result)
                    node.set_child(leaf_node)
                    print('无样本，判断为',most_common_label)
                else:
                    print(node.attribute,'大于',node.split_point,'样本个数：',len(X_subset))
                    childnode = build_decision_tree(X_subset, y_subset)
                    childnode.set_branchvalue('>'+str(best_split_point))
                    print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                    node.set_child(childnode)
    return node


# 预剪枝决策树代码        
def build_decision_tree_prun(X, y, X_valid, y_valid):
    global label_count 
    if len(set(y)) == 1:  # 所有样本属于同一类别，创建叶子节点
        leaf_node = DecisionTreeNode()
        leaf_node.set_result(y.iloc[0])
        return leaf_node
    
    if len(X.columns) == 0:  # 所有属性已经用完，创建叶子节点，选择最多的类别作为结果
        leaf_node = DecisionTreeNode()
        most_common_label = y.value_counts().idxmax()
        leaf_node.set_result(most_common_label)
        return leaf_node

    
    
     # 选取最佳划分属性
    best_info_gain = -np.inf
    best_attribute = None
    best_split_point = None
    best_attribute,best_split_point,best_info_gain = find_best_attribute(X, y)
    if best_info_gain < 0:  # 信息增益小于0，创建叶子节点，选择最多的类别作为结果
        leaf_node = DecisionTreeNode()
        most_common_label = y.value_counts().idxmax()
        leaf_node.set_result(most_common_label)
        return leaf_node

    
    
    node = DecisionTreeNode()
    node.set_attribute(best_attribute)
    node.set_split_point(best_split_point)
    node.add_count()
    node.set_fake_result(y.value_counts().idxmax())

    if not node.is_leaf():
        label_count += 1
    
    
    #预剪枝,验证集不为空且所有样本不属于同一类别
    if(is_pre_pruning(X,y,X_valid, y_valid,best_attribute,best_split_point,node)):
        #剪枝
        print("剪枝")
        most_common_label = y.value_counts().idxmax()#返回最多的类别
        node.set_result(most_common_label)#设置叶子节点的结果
        return node
    else:
        #不剪枝
        pass
    
    
    # 递归构建子树
    if best_split_point is None:  # 处理离散属性
        for value in X[best_attribute].unique():
            mask = X[best_attribute] == value# 选出X中等于value的样本划分子树
            mask_valid = X_valid[best_attribute] == value
            X_subset = X[mask]
            y_subset = y[mask]
            X_valid_subset = X_valid[mask_valid]
            y_valid_subset = y_valid[mask_valid]
            print(X_subset,y_subset)
            if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                leaf_node = DecisionTreeNode()
                most_common_label = y.value_counts().idxmax()#返回最多的类别
                leaf_node.set_result(most_common_label)#设置叶子节点的结果
                leaf_node.set_branchvalue(value)
                node.set_child(leaf_node)#设置左子树
            else:
                print(X_valid_subset,y_valid_subset)
                childnode = build_decision_tree_prun(X_subset, y_subset,X_valid_subset,y_valid_subset)#递归创建左子树
                childnode.set_branchvalue(value)
                print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                node.set_child(childnode)#递归创建左子树
    else:  # 处理连续属性
            #for循环分别处理大于和小于两个子树
        for num in range(2):
            if num == 0:
                #小于等于分割点的部分
                mask = X[best_attribute] <= best_split_point
                mask_valid = X_valid[best_attribute] <= best_split_point
                X_subset = X[mask]
                y_subset = y[mask]
                X_valid_subset = X_valid[mask_valid]
                y_valid_subset = y_valid[mask_valid]
                if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                    leaf_node = DecisionTreeNode()
                    most_common_label = y.value_counts().idxmax()
                    leaf_node.set_result(most_common_label)
                    leaf_node.set_branchvalue('<='+str(best_split_point))
                    print(leaf_node.branchvaule)
                    node.set_child(leaf_node)
                    print('无样本，判断为',most_common_label)
                else:
                    print(node.attribute,'小于等于',node.split_point,'样本个数：',len(X_subset),'验证集样本个数：',len(X_valid_subset))
                    childnode = build_decision_tree_prun(X_subset, y_subset,X_valid_subset,y_valid_subset)
                    childnode.set_branchvalue('<='+str(best_split_point))
                    print(childnode.branchvaule)
                    node.set_child(childnode)
            if num == 1:
                #大于分割点的部分
                mask = X[best_attribute] > best_split_point
                mask_valid = X_valid[best_attribute] > best_split_point
                X_subset = X[mask]
                y_subset = y[mask]    
                if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                    leaf_node = DecisionTreeNode()
                    most_common_label = y.value_counts().idxmax()
                    leaf_node.set_result(most_common_label)
                    leaf_node.set_branchvalue('>'+str(best_split_point))
                    print(leaf_node.branchvaule)
                    node.set_child(leaf_node)
                    print('无样本，判断为',most_common_label)
                else:
                    print(node.attribute,'大于',node.split_point,'样本个数：',len(X_subset),'验证集样本个数：',len(X_valid_subset))
                    childnode = build_decision_tree_prun(X_subset, y_subset,X_valid_subset,y_valid_subset)
                    childnode.set_branchvalue('>'+str(best_split_point))
                    print(childnode.branchvaule)
                    node.set_child(childnode)
    return node


#信息增益率预剪枝决策树代码
def build_decision_tree_prun_rate(X, y, X_valid, y_valid):
    global label_count 
    if len(set(y)) == 1:  # 所有样本属于同一类别，创建叶子节点
        leaf_node = DecisionTreeNode()
        leaf_node.set_result(y.iloc[0])
        return leaf_node
    
    if len(X.columns) == 0:  # 所有属性已经用完，创建叶子节点，选择最多的类别作为结果
        leaf_node = DecisionTreeNode()
        most_common_label = y.value_counts().idxmax()
        leaf_node.set_result(most_common_label)
        return leaf_node

    
    
     # 选取最佳划分属性
    best_info_gain_rate = -np.inf
    best_attribute = None
    best_split_point = None
    best_attribute,best_split_point,best_info_gain_rate = find_best_attribute2(X, y)
    if best_info_gain_rate < 0:  # 信息增益小于0，创建叶子节点，选择最多的类别作为结果
        leaf_node = DecisionTreeNode()
        most_common_label = y.value_counts().idxmax()
        leaf_node.set_result(most_common_label)
        return leaf_node

    
    
    node = DecisionTreeNode()
    node.set_attribute(best_attribute)
    node.set_split_point(best_split_point)
    node.add_count()
    node.set_fake_result(y.value_counts().idxmax())

    if not node.is_leaf():
        label_count += 1
    
    
    #预剪枝,验证集不为空且所有样本不属于同一类别
    if(is_pre_pruning(X,y,X_valid, y_valid,best_attribute,best_split_point,node)):
        #剪枝
        print("剪枝")
        most_common_label = y.value_counts().idxmax()#返回最多的类别
        node.set_result(most_common_label)#设置叶子节点的结果
        return node
    else:
        #不剪枝
        pass
    
    
    # 递归构建子树
    if best_split_point is None:  # 处理离散属性
        for value in X[best_attribute].unique():
            mask = X[best_attribute] == value# 选出X中等于value的样本划分子树
            mask_valid = X_valid[best_attribute] == value
            X_subset = X[mask]
            y_subset = y[mask]
            X_valid_subset = X_valid[mask_valid]
            y_valid_subset = y_valid[mask_valid]
            print(X_subset,y_subset)
            if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                leaf_node = DecisionTreeNode()
                most_common_label = y.value_counts().idxmax()#返回最多的类别
                leaf_node.set_result(most_common_label)#设置叶子节点的结果
                leaf_node.set_branchvalue(value)
                node.set_child(leaf_node)#设置左子树
            else:
                print(X_valid_subset,y_valid_subset)
                childnode = build_decision_tree_prun(X_subset, y_subset,X_valid_subset,y_valid_subset)#递归创建左子树
                childnode.set_branchvalue(value)
                print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                node.set_child(childnode)#递归创建左子树
    else:  # 处理连续属性
            #for循环分别处理大于和小于两个子树
        for num in range(2):
            if num == 0:
                #小于等于分割点的部分
                mask = X[best_attribute] <= best_split_point
                mask_valid = X_valid[best_attribute] <= best_split_point
                X_subset = X[mask]
                y_subset = y[mask]
                X_valid_subset = X_valid[mask_valid]
                y_valid_subset = y_valid[mask_valid]
                if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                    leaf_node = DecisionTreeNode()
                    most_common_label = y.value_counts().idxmax()
                    leaf_node.set_result(most_common_label)
                    leaf_node.set_branchvalue('<='+str(best_split_point))
                    print(leaf_node.branchvaule)
                    node.set_child(leaf_node)
                    print('无样本，判断为',most_common_label)
                else:
                    print(node.attribute,'小于等于',node.split_point,'样本个数：',len(X_subset),'验证集样本个数：',len(X_valid_subset))
                    childnode = build_decision_tree_prun(X_subset, y_subset,X_valid_subset,y_valid_subset)
                    childnode.set_branchvalue('<='+str(best_split_point))
                    print(childnode.branchvaule)
                    node.set_child(childnode)
            if num == 1:
                #大于分割点的部分
                mask = X[best_attribute] > best_split_point
                mask_valid = X_valid[best_attribute] > best_split_point
                X_subset = X[mask]
                y_subset = y[mask]    
                if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                    leaf_node = DecisionTreeNode()
                    most_common_label = y.value_counts().idxmax()
                    leaf_node.set_result(most_common_label)
                    leaf_node.set_branchvalue('>'+str(best_split_point))
                    print(leaf_node.branchvaule)
                    node.set_child(leaf_node)
                    print('无样本，判断为',most_common_label)
                else:
                    print(node.attribute,'大于',node.split_point,'样本个数：',len(X_subset),'验证集样本个数：',len(X_valid_subset))
                    childnode = build_decision_tree_prun(X_subset, y_subset,X_valid_subset,y_valid_subset)
                    childnode.set_branchvalue('>'+str(best_split_point))
                    print(childnode.branchvaule)
                    node.set_child(childnode)
    return node


后剪枝

In [ ]:
#后剪枝

#预测函数
def predict_tree_single(node, x):
    if node.is_leaf():
        return node.result
    else:
        #如果是连续值属性
        if node.split_point is not None:
            if x[node.attribute] <= node.split_point:
                return predict_tree_single(node.children[0], x)
            else:
                return predict_tree_single(node.children[1], x)
        else:#离散值属性
            for child in node.children:
                if child.branchvaule == x[node.attribute]:
                    return predict_tree_single(child, x)
                
#生成预测结果
def generate_post_y(decision_tree, X):
    y_pred = []
    for i in range(len(X)):
        x = X.iloc[i]
        y_pred.append(predict_tree_single(decision_tree, x))
    return np.array(y_pred)


def calculate_accuracy(node, X_val, y_val):
    # 使用从给定节点开始的树进行预测
    predictions = generate_post_y(node, X_val)

    # 计算准确性
    correct_predictions = sum(1 for p, true_label in zip(predictions, y_val) if p == true_label)
    return correct_predictions / len(y_val)


def find_max_label_nodes(node):
    # 广度优先搜索找到具有最大label值的节点
    max_label_nodes = []

    queue = [node]
    while queue:
        current_node = queue.pop(0)
        if not current_node.is_leaf():
            max_label_nodes.append(current_node)

        queue.extend(child for child in current_node.children if not child.is_leaf())

    # 按label值从大到小排序
    max_label_nodes.sort(key=lambda x: x.label, reverse=True)

    return max_label_nodes

def post_pruning(decisiontree, X_val, y_val):
    processed_nodes = set()

    max_label_nodes = find_max_label_nodes(decisiontree)

    for node in max_label_nodes:
        if node in processed_nodes:
            continue

        # 计算剪枝前的验证集准确性
        accuracy_before = calculate_accuracy(decisiontree, X_val, y_val)
        print("剪枝前的准确率：", accuracy_before)

        # 尝试剪枝
        node.result = node.fake_result

        # 计算剪枝后的验证集准确性
        accuracy_after = calculate_accuracy(decisiontree, X_val, y_val)
        print("假设剪枝后的准确率：", accuracy_after)

        # 恢复原始结果值
        node.result = None

        # 比较准确性并进行剪枝
        if accuracy_after >= accuracy_before:
            print("剪枝, 该节点label值为：", node.label)
            node.set_result(node.fake_result)  # 剪枝
            node.set_attribute(None)
            processed_nodes.add(node)
        else:
            print("不剪枝, 该节点label值为：", node.label)
            processed_nodes.add(node)

# 使用示例
# decisiontree 是树的根节点

            


可视化

In [ ]:

import graphviz
dot = graphviz.Digraph()


parent = None
leaf_counter = 0

def add_nodes(node):
    global parent
    global leaf_counter
    if node.is_leaf():
        leaf_name = str(node.result + str(leaf_counter))
        dot.node(name=leaf_name,fontname ="SimHei") 
        dot.edge(tail_name=parent, head_name=leaf_name, label=str(node.branchvaule),fontname ="SimHei")
        leaf_counter += 1
        return
    else:
        dot.node(name=str(node.attribute),fontname ="SimHei")
        for child in node.children:
            parent = node.attribute 
            #如果孩子是非叶子节点
            if not child.is_leaf():
                dot.edge(tail_name=parent, head_name=str(child.attribute), label=str(child.branchvaule),fontname ="SimHei")
                add_nodes(child)
            if child.is_leaf():
                leaf_name = str(child.result + str(leaf_counter))
                dot.node(name=leaf_name,fontname ="SimHei") 
                dot.edge(tail_name=parent, head_name=leaf_name, label=str(child.branchvaule),fontname ="SimHei")
                leaf_counter += 1
            parent = node.attribute
    leaf_counter = 0
    return
# add_nodes(decision_tree)
# dot.render('decision_tree',format='png')
# dot.clear()
# post_pruning(decision_tree, X_test, y_test)
# add_nodes(decision_tree)
# dot.render('decision_tree_postpruning',format='png')

准确率比较:信息增益树

In [ ]:
#未剪枝
dot.clear()
decision_tree = build_decision_tree(X_train, y_train)
print('未剪枝准确率:',calculate_accuracy(decision_tree, X_test, y_test))
add_nodes(decision_tree)
dot.render('tree',format='png')
dot.clear()


    色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率
12  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  0.639  0.161
14  乌黑  稍蜷  浊响  清晰  稍凹  软粘  0.360  0.370
0   青绿  蜷缩  浊响  清晰  凹陷  硬滑  0.697  0.460
10  浅白  硬挺  清脆  模糊  平坦  硬滑  0.245  0.057
3   青绿  蜷缩  沉闷  清晰  凹陷  硬滑  0.608  0.318
2   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  0.634  0.264
1   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  0.774  0.376
13  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  0.657  0.198
6   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  0.481  0.149
16  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  0.719  0.103
11  浅白  蜷缩  浊响  模糊  平坦  软粘  0.343  0.099 12    否
14    否
0     是
10    否
3     是
2     是
1     是
13    否
6     是
16    否
11    否
Name: 好瓜, dtype: object
当前最佳划分属性： 纹理 当前标签值： 0 当前假设结果： 否
    色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率
12  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  0.639  0.161
13  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  0.657  0.198
6   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  0.481  0.149
16  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  0.719  0.103 12    否
13    否
6     是
16    否
Name: 好瓜, dtype: object
当前最佳划分属性： 色泽 当前标签值： 1 当前假设结果： 否
青绿 2 是否为好瓜: 否
浅白 1 是否为好瓜: 否
乌黑 1 是否为好瓜: 是
稍糊 4 是否为好瓜: None
    色泽  根蒂  

In [ ]:

#预剪枝
decision_tree_prun =  build_decision_tree_prun(X_train, y_train,X_test, y_test)
print('预剪枝准确率:',calculate_accuracy(decision_tree_prun, X_test, y_test))
add_nodes(decision_tree_prun)
dot.render('tree_prun',format='png')
dot.clear()

划分前的结果： 否
划分前的准确率： 0.5
['否' '是' '是' '是' '是' '否']
离散属性划分后的准确率： 0.16666666666666666
剪枝
预剪枝准确率: 0.5


In [ ]:
#后剪枝
post_pruning(decision_tree, X_test, y_test)
print('后剪枝准确率:',calculate_accuracy(decision_tree, X_test, y_test))
add_nodes(decision_tree)
dot.render('tree_postpruning',format='png')
dot.clear()

剪枝前的准确率： 0.3333333333333333
假设剪枝后的准确率： 0.6666666666666666
剪枝, 该节点label值为： 2
剪枝前的准确率： 0.6666666666666666
假设剪枝后的准确率： 0.8333333333333334
剪枝, 该节点label值为： 1
剪枝前的准确率： 0.8333333333333334
假设剪枝后的准确率： 0.5
不剪枝, 该节点label值为： 0
后剪枝准确率: 0.8333333333333334


In [ ]:
print('未剪枝准确率:',calculate_accuracy(decision_tree, X_test, y_test))
print('预剪枝准确率:',calculate_accuracy(decision_tree_prun, X_test, y_test))
print('后剪枝准确率:',calculate_accuracy(decision_tree, X_test, y_test))

未剪枝准确率: 0.8333333333333334
预剪枝准确率: 0.5
后剪枝准确率: 0.8333333333333334


准确率比较:信息增益率树


In [ ]:
#未剪枝
dot.clear()
decision_tree_rate = build_decision_tree_rate(X_train, y_train)
print('未剪枝准确率:',calculate_accuracy(decision_tree_rate, X_test, y_test))
add_nodes(decision_tree_rate)
dot.render('rate_tree',format='png')
dot.clear()

    色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率
12  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  0.639  0.161
14  乌黑  稍蜷  浊响  清晰  稍凹  软粘  0.360  0.370
0   青绿  蜷缩  浊响  清晰  凹陷  硬滑  0.697  0.460
10  浅白  硬挺  清脆  模糊  平坦  硬滑  0.245  0.057
3   青绿  蜷缩  沉闷  清晰  凹陷  硬滑  0.608  0.318
2   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  0.634  0.264
1   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  0.774  0.376
13  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  0.657  0.198
6   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  0.481  0.149
16  青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  0.719  0.103
11  浅白  蜷缩  浊响  模糊  平坦  软粘  0.343  0.099 12    否
14    否
0     是
10    否
3     是
2     是
1     是
13    否
6     是
16    否
11    否
Name: 好瓜, dtype: object
当前最佳划分属性： 密度 当前标签值： 4 当前假设结果： 否
密度 小于等于 0.4205 样本个数： 3
<=0.4205 3 是否为好瓜: 否
密度 大于 0.4205 样本个数： 8
    色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率
12  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  0.639  0.161
0   青绿  蜷缩  浊响  清晰  凹陷  硬滑  0.697  0.460
3   青绿  蜷缩  沉闷  清晰  凹陷  硬滑  0.608  0.318
2   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  0.634  0.264
1   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  0.774  0.376
13  浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  0.657  0.198
6   乌黑  稍蜷  浊

In [ ]:
#预剪枝
decision_tree_prun_rate =  build_decision_tree_prun_rate(X_train, y_train,X_test, y_test)
print('预剪枝准确率:',calculate_accuracy(decision_tree_prun_rate, X_test, y_test))
add_nodes(decision_tree_prun_rate)
dot.render('rate_tree_prun',format='png')
dot.clear()

划分前的结果： 否
划分前的准确率： 0.5
['否' '否' '是' '是' '是' '是']
连续属性划分后的准确率： 0.5
剪枝
预剪枝准确率: 0.5


In [ ]:
#后剪枝
post_pruning(decision_tree_rate, X_test, y_test)
print('后剪枝准确率:',calculate_accuracy(decision_tree, X_test, y_test))
add_nodes(decision_tree_rate)
dot.render('rate_tree_postpruning',format='png')
dot.clear()

剪枝前的准确率： 0.5
假设剪枝后的准确率： 0.6666666666666666
剪枝, 该节点label值为： 6
剪枝前的准确率： 0.6666666666666666
假设剪枝后的准确率： 0.5
不剪枝, 该节点label值为： 5
剪枝前的准确率： 0.6666666666666666
假设剪枝后的准确率： 0.5
不剪枝, 该节点label值为： 4
后剪枝准确率: 0.8333333333333334


In [ ]:
print('未剪枝准确率:',calculate_accuracy(decision_tree_rate, X_test, y_test))
print('预剪枝准确率:',calculate_accuracy(decision_tree_prun_rate, X_test, y_test))
print('后剪枝准确率:',calculate_accuracy(decision_tree_rate, X_test, y_test))

未剪枝准确率: 0.6666666666666666
预剪枝准确率: 0.5
后剪枝准确率: 0.6666666666666666


选做

In [ ]:
data = pd.read_csv('penguins.csv') 

X = data.iloc[:,1:-1]  
y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
# print(X_train)
# print(y_train)
# print(X_test)
# print(y_test)
#打印每种属性的种类
# for col in X.columns:
#     print(col,":",col.type)

def build_decision_tree_penguins(X, y):
    global label_count 
    if len(set(y)) == 1:  # 所有样本属于同一类别，创建叶子节点
        leaf_node = DecisionTreeNode()
        leaf_node.set_result(y.iloc[0])
        return leaf_node
    
    if len(X.columns) == 0:  # 所有属性已经用完，创建叶子节点，选择最多的类别作为结果
        leaf_node = DecisionTreeNode()
        most_common_label = y.value_counts().idxmax()
        leaf_node.set_result(most_common_label)
        return leaf_node
    
    # 选取最佳划分属性
    best_info_gain = -np.inf
    best_attribute = None
    best_split_point = None
    best_attribute,best_split_point,best_info_gain = find_best_attribute(X, y)
    if best_info_gain < 0:  # 信息增益小于0，创建叶子节点，选择最多的类别作为结果
        leaf_node = DecisionTreeNode()
        most_common_label = y.value_counts().idxmax()
        leaf_node.set_result(most_common_label)
        return leaf_node
    
    node = DecisionTreeNode()
    node.set_attribute(best_attribute)
    node.set_split_point(best_split_point)
    node.add_count()
    node.set_fake_result(y.value_counts().idxmax())
    
    if not node.is_leaf():
        label_count += 1
        
    print(X,y)
    print("当前最佳划分属性：",node.attribute,"当前标签值：",node.label,"当前假设结果：",node.fake_result)
    
    # 递归构建子树
    if best_split_point is None:  # 处理离散属性
        for value in X[best_attribute].unique():
            mask = X[best_attribute] == value# 选出X中等于value的样本划分子树
            X_subset = X[mask]
            y_subset = y[mask]
            #分完后将该属性从X中删除
            X_subset = X_subset.drop(columns=[best_attribute])
            if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                leaf_node = DecisionTreeNode()
                most_common_label = y.value_counts().idxmax()#返回最多的类别
                leaf_node.set_result(most_common_label)#设置叶子节点的结果
                leaf_node.set_branchvalue(value)
                print(leaf_node.branchvaule,0)
                node.set_child(leaf_node)#设置子树
            else:
                childnode = build_decision_tree(X_subset, y_subset)#递归创建左子树
                childnode.set_branchvalue(value)
                print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                node.set_child(childnode)#递归创建子树
    else:  # 处理连续属性
            #for循环分别处理大于和小于两个子树
        for num in range(2):
            if num == 0:
                #小于等于分割点的部分
                mask = X[best_attribute] <= best_split_point
                X_subset = X[mask]
                y_subset = y[mask]
                #分完后将该属性从X中删除
                if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                    leaf_node = DecisionTreeNode()
                    most_common_label = y.value_counts().idxmax()
                    leaf_node.set_result(most_common_label)
                    leaf_node.set_branchvalue('<='+str(best_split_point))
                    # print(leaf_node.branchvaule)
                    node.set_child(leaf_node)
                    print('无样本，判断为',most_common_label)
                else:
                    X_subset = X_subset.drop(columns=[best_attribute])
                    print(node.attribute,'小于等于',node.split_point,'样本个数：',len(X_subset))
                    childnode = build_decision_tree(X_subset, y_subset)
                    childnode.set_branchvalue('<='+str(best_split_point))
                    # print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                    node.set_child(childnode)
            if num == 1:
                #大于分割点的部分
                mask = X[best_attribute] > best_split_point
                X_subset = X[mask]
                y_subset = y[mask]
                if len(X_subset) == 0:  # 无样本，创建叶子节点，选择最多的类别作为结果
                    leaf_node = DecisionTreeNode()
                    most_common_label = y.value_counts().idxmax()
                    leaf_node.set_result(most_common_label)
                    leaf_node.set_branchvalue('>'+str(best_split_point))
                    print(leaf_node.branchvaule,leaf_node.result)
                    node.set_child(leaf_node)
                    print('无样本，判断为',most_common_label)
                else:
                    print(node.attribute,'大于',node.split_point,'样本个数：',len(X_subset))
                    X_subset = X_subset.drop(columns=[best_attribute])
                    childnode = build_decision_tree(X_subset, y_subset)
                    childnode.set_branchvalue('>'+str(best_split_point))
                    # print(childnode.branchvaule,len(X_subset),"是否为好瓜:",childnode.result)
                    node.set_child(childnode)
    return node

In [ ]:
label_count = 0
penguins_tree = build_decision_tree_penguins(X_train, y_train)

print('未剪枝准确率:',calculate_accuracy(penguins_tree, X_test, y_test))
add_nodes(penguins_tree)
dot.render('penguins_tree',format='png')
dot.clear()

     bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g     sex  \
292           331.1          310.7              473.0       4042.0  female   
298           337.1          316.7              479.0       4048.0  female   
106           145.1          124.7              287.0       3856.0    male   
47             86.1           65.7              228.0       3797.0  female   
77            116.1           95.7              258.0       3827.0  female   
..              ...            ...                ...          ...     ...   
130           169.1          148.7              311.0       3880.0    male   
241           280.1          259.7              422.0       3991.0  female   
253           292.1          271.7              434.0       4003.0  female   
155           194.1          173.7              336.0       3905.0    male   
331           370.1          349.7              512.0       4081.0    male   

     year  
292  2007  
298  2007  
106  2009  
47   2007  
77 